In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binomtest # Imported for the two-sided paired sign test.
from scipy.stats import kruskal # Import for the Kruskal-Wallis test.
import scikit_posthocs as sp # Import Dunn's test:

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Phase 1: Determining significance in county-level MMR vaccination trends across states using a Paired Sign Test.

In [2]:
''' Step 1: Import county-level MMR rate data from Dong et al., 2025.
    Github Repository: https://github.com/CSSEGISandData/MMR_data
    Citation: Dong E, Saiyed S, Nearchou A, Okura Y, Gardner LM. Trends in County-Level MMR Vaccination Coverage in Children in the United States. JAMA. Published online June 02, 2025. doi:10.1001/jama.2025.8952
'''

mmr_data_us_counties = pd.read_csv('mmr_data_us_counties.csv') # Data from Dong et al.

''' Step 2: Process data for pre pandemic and post pandemic periods. Consistent with Dong et al.,
    we define pre pandemic MMR rates as the average of the annual county-level rates for the 
    2017-18, 2018-19, and 2019-20 school years, while post pandemic rates are equivalently defined
    for the 2022-23 and 2023-24 school years.'''

# MMR_pre_covid = average of SY2017_18, SY2018_19, SY2019_20
mmr_data_us_counties["MMR_pre_covid"] = mmr_data_us_counties[["SY2017_18", "SY2018_19", "SY2019_20"]].mean(axis=1, skipna=True)

# MMR_post_covid = average of SY2022_23, SY2023_24
mmr_data_us_counties["MMR_post_covid"] = mmr_data_us_counties[["SY2022_23", "SY2023_24"]].mean(axis=1, skipna=True)

# display(mmr_data_us_counties) # Uncomment to display results for step 2.

''' Step 3: As per Dong et al., 2025, exclude states that do not have comprehensive 
    county-level MMR vaccination rate data as per Dong et al., 2025 Supplement 2: eTable. MMR 
    Reporting by State.'''

# States without comprehensive county-level data:
county_data_unavailable = ['AK', 'DE', 'ID', 'MT', 'NE', 'NH', 'OH', 'WV', 'AR', 'DC', 
                           'GA', 'IL', 'IN', 'WY', 'NM', 'LA', 'MS', 'ND']

# Filter mmr_data_us_counties dataframe to exclude states:
mmr_data_us_counties = mmr_data_us_counties[~mmr_data_us_counties['State'].isin(county_data_unavailable)]

''' Step 4: Drop any counties within included states that do not have pre or post pandemic data. 
    This will get us down to the 2066 counties analyzed by Dong et al., 2025'''

mmr_data_us_counties.dropna(subset=["MMR_pre_covid", "MMR_post_covid"], inplace = True)
mmr_data_us_counties = mmr_data_us_counties.reset_index(drop = True)
# display(mmr_data_us_counties) # Uncomment to display results for step 4.

''' Step 5: Now that our data is loaded, we can implement the paired sign test to identify which 
    states experience statistically significant county-level MMR rate changes. We conduct this 
    test using scipy.stats binomtest.'''

results = []
for state, subset in mmr_data_us_counties.groupby("State"):
    # Generate two pairs of observations for each state.
    prepandemic = subset["MMR_pre_covid"]
    postpandemic = subset["MMR_post_covid"]
    
    # Calculate the difference between the observations.
    difference = (postpandemic - prepandemic)
    non_tied_differences = difference[difference != 0]
    
    # Conduct paired sign test:
    n_positive = np.sum(non_tied_differences > 0) # Count positive differences.
    n_total = len(non_tied_differences) # Count total differences.
    if n_total > 0:
        paired_sign_test = binomtest(n_positive, n_total, p = 0.5, alternative = "two-sided").pvalue # Two sided sign test.
    else:
        paired_sign_test = np.nan
    
    results.append({
        "State" : state,
        "n_counties" : len(postpandemic),
        "n_positive" : n_positive,
        "paired_sign_test" : paired_sign_test
    })
    
sign_test_results = pd.DataFrame(results)
display(sign_test_results) # Display paired sign test results:

,State,n_counties,n_positive,paired_sign_test
0,AL,67,19,5.216128e-04
1,AZ,15,0,6.103516e-05
2,CA,57,51,5.676193e-10
3,CO,64,27,2.604355e-01
4,CT,8,8,7.812500e-03
5,FL,67,9,6.811498e-10
6,HI,4,0,1.250000e-01
7,IA,99,17,2.180641e-11
8,KS,105,20,4.294849e-10
9,KY,120,41,6.667239e-04


### Phase 2: Kruskal-Wallis test and Dunn's post-hoc test (Holm Correction) to Determine Significance of State-Level NME Policy on State-Level MMR Vaccination Trends

In [3]:
''' Step 1: Import CDC MMR Rate Data from School Vaxview. Provided at the following link:
    https://www.cdc.gov/schoolvaxview/data/index.html
    Citation: 6. CDC. Vaccination Coverage and Exemptions among Kindergartners. https://www.cdc.gov/schoolvaxview/data/index.html 
'''

CDC_vaccination_data = pd.read_csv('Vaccination_Coverage_and_Exemptions_among_Kindergartners_20250625.csv') # Update filename with your downloaded file.
CDC_vaccination_data

''' Step 2: Filter data to only included MMR rates for the pre and post pandemic period. Consistent 
    with Dong et al., we define pre pandemic MMR rates as the average of the annual county-level rates 
    for the 2017-18, 2018-19, and 2019-20 school years, while post pandemic rates are equivalently defined
    for the 2022-23 and 2023-24 school years.'''

# Filter for MMR data only.
CDC_vaccination_data = CDC_vaccination_data[CDC_vaccination_data['Vaccine/Exemption'] == 'MMR']

# Convert "Estimation (%)" column to a numeric value:
CDC_vaccination_data["Estimate (%)"] = pd.to_numeric(CDC_vaccination_data["Estimate (%)"], errors="coerce")

# Define pre pandemic and post pandemic years:
prepandemic_years  = ["2017-18", "2018-19", "2019-20"]
postpandemic_years = ["2022-23", "2023-24"]
years_total = prepandemic_years + postpandemic_years

# Process data per state:
processed = []
for state, period in CDC_vaccination_data.groupby("Geography"):
    # Define pre pandemic and post pandemic averages:
    prepandemic_average = period[period["School Year"].isin(prepandemic_years)]["Estimate (%)"].mean()
    postpandemic_average = period[period["School Year"].isin(postpandemic_years)]["Estimate (%)"].mean()
    
    # Process into one DataFrame:
    processed.append({
        "Vaccine":                period["Vaccine/Exemption"].iat[0],
        "Geography Type":         period["Geography Type"].iat[0],
        "State":                  state,
        "MMR_pre_covid":          prepandemic_average / 100,
        "MMR_post_covid":         postpandemic_average / 100,
    })
    
CDC_data_processed = pd.DataFrame(processed) # Set results into a new DataFrame.

''' Step 3: Import non-medical exemption data, compiled from the CDC and NCSL. Data is 
    provided in .csv format in GitHub. The 3 NME categories are classified as follows:
    NME 0: No non-medical exemption options (Medical Exemption Only)
    NME 1: Religious non-medical exemption options only.
    NME 2: Religious and philosophical/personal non-medical exemption options.
    Citations:
    1. CDC. Public Health Law. State School Immunization Requirements and Vaccine Exemption Laws https://www.cdc.gov/phlp/docs/school-vaccinations.pdf.
    2. State Non-Medical Exemptions from School Immunization Requirements. https://www.ncsl.org/health/state-non-medical-exemptions-from-school-immunization-requirements.
'''
NME_data = pd.read_csv("NME_compiled.csv") # open the data.

# Merge the data into main dataframe.
CDC_data_processed = (CDC_data_processed.merge(NME_data, left_on = "State", 
                                               right_on = 'State_Name', how = "left"))

# Only keep relevant columns:
CDC_data_processed = CDC_data_processed[["State", "State_Abbr", "MMR_pre_covid",
                                         "MMR_post_covid", "nme_pre", "nme_post"]]

''' Step 4: Clean data. Remove Montana (lack of post pandemic data), D.C. (lack of pre pandemic data),
    and NY-City of New York, TX-City of Houston, U.S. Median, United States (not state-level).'''

state_data_unavailable = ["Montana", "District of Columbia", "U.S. Median", "United States",
                          "NY-City of New York", "TX-City of Houston"]

# Filter mmr_data_us_counties dataframe to exclude states:
CDC_data_processed = CDC_data_processed[~CDC_data_processed['State'].isin(state_data_unavailable)]
CDC_data_processed = CDC_data_processed.reset_index(drop = True)
# display(CDC_data_processed) # Uncomment to display results for step 4. 

''' Step 5: Calculate the difference between pre pandemic to post pandemic periods:
    (post pandemic - pre pandemic)'''
CDC_data_processed['Difference'] = CDC_data_processed['MMR_post_covid'] - CDC_data_processed['MMR_pre_covid']
display(CDC_data_processed) # Display data processing results for step 5.

,State,State_Abbr,MMR_pre_covid,MMR_post_covid,nme_pre,nme_post,Difference
0,Alabama,AL,0.899667,0.9385,1.0,1.0,0.038833
1,Alaska,AK,0.916000,0.8395,1.0,1.0,-0.076500
2,Arizona,AZ,0.930333,0.8960,2.0,2.0,-0.034333
3,Arkansas,AR,0.934667,0.9220,2.0,2.0,-0.012667
4,California,CA,0.966333,0.9635,0.0,0.0,-0.002833
5,Colorado,CO,0.890667,0.8765,2.0,2.0,-0.014167
6,Connecticut,CT,0.962000,0.9750,1.0,0.0,0.013000
7,Delaware,DE,0.972500,0.9445,1.0,1.0,-0.028000
8,Florida,FL,0.936667,0.8935,1.0,1.0,-0.043167
9,Georgia,GA,0.933667,0.8825,1.0,1.0,-0.051167


In [5]:
''' Step 6: Perform Kruskal-Wallis Test using scipy.stats. Evaluate if there is a statistical
    difference between states with NME 0, NME 1, and NME 2.'''

# Create 3 groups:
group0 = CDC_data_processed.loc[CDC_data_processed['nme_post'] == 0, 'Difference'].dropna()
group1 = CDC_data_processed.loc[CDC_data_processed['nme_post'] == 1, 'Difference'].dropna()
group2 = CDC_data_processed.loc[CDC_data_processed['nme_post'] == 2, 'Difference'].dropna()

# Perform Kruskal-Wallis H-test:
h_stat, p_value = kruskal(group0, group1, group2)
h_stat = h_stat.round(3) # Round to an appropriate number of decimals.
p_value = p_value.round(4) # Round to an appropriate number of decimals.

print(f"Kruskal-Wallis test result: H = {h_stat}, p-value = {p_value}")
print("\nKruskal-Wallis Results: Since p-value < 0.05, we confirm that there is a statistically significant difference between at least one group. Therefore, a post-hoc test can be conducted.")

''' Step 7: Perform a follow-up Dunn's test with Holm correction to determine the pairwise difference
    between groups, thus determining which groups are significantly different.'''
# Group our three samples together:
samples = [group0, group1, group2]

# Perform Dunn's test with Holm correction:
p_values = sp.posthoc_dunn(samples, p_adjust = 'holm')

# Display pairwise comparisons based on results:
p_values.index = p_values.columns = ['NME 0', 'NME 1', 'NME 2']
p_values = p_values.round(3) # Round to an appropriate number of decimals.
print("\nResults of the pairwise Dunn's test (Holm correction):")
display(p_values) # Display results for Dunn's test with Holm correction.

Kruskal-Wallis test result: H = 11.905, p-value = 0.0026

Kruskal-Wallis Results: Since p-value < 0.05, we confirm that there is a statistically significant difference between at least one group. Therefore, a post-hoc test can be conducted.

Results of the pairwise Dunn's test (Holm correction):


,NME 0,NME 1,NME 2
NME 0,1.000,0.034,0.002
NME 1,0.034,1.000,0.068
NME 2,0.002,0.068,1.000
